In [6]:
# Importation des libraries/packages de base. 

import pandas as pd
import os
import numpy as np
import dask.dataframe as dd 
import warnings 
warnings.filterwarnings("ignore")



In [7]:
# Fichier avec les variables d'intéret : dernière version : 21/02 

df_cct = pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/df_all.parquet")

In [8]:
#creation de 2000 parquets (1 par user)
for k in range (2000):
    df=df_cct[df_cct['User']==k]
    df.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all'+ str(k)+'.parquet')



In [24]:
from sklearn import preprocessing
#fonction qui retire les colonnes inutiles, qui cree la colonne 'Number_year_since_PIN_last_Changed' qui
    #qui cree des dummies a partir des variable categorielle et qui normalise les variables non-categorielles

def modification_data_frame(df_cct):
    
    #On retire les colonnes qui ne sont plus utiles
    df_cct =  df_cct.drop([ "Use Chip", 'day_of_week','Card', 'Month', 'Day', 'Hours'], axis = 1)
    
    
    #pour ne pas avoir de soucis lors de la concatenations des dataframe (df_cct et dummies), on reinitialise les index
    df_cct=df_cct.reset_index()    
    df_cct =  df_cct.drop(["index"], axis = 1)

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    features_array = enc.fit_transform(df_cct[['Card Brand']]).toarray()
    features_labels = np.hstack(np.array(enc.categories_))
    df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels).reset_index()], axis = 1)

    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    features_array = enc.fit_transform(df_cct[['Card Type']]).toarray()
    features_labels = np.hstack(np.array(enc.categories_))
    df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

    df_cct =  df_cct.drop(["index","Card Brand",'Card Type'], axis = 1)
    
    
    
    #On centre et reduit les colonnes a normaliser suivantes:
    Normalizing_List=['Amount', 'Credit Limit', 'delta_t_s', 'delta_t_s_card',
       'amt/daily_income', 'daily_amount', 'nb_daily_declines_card',
       'nb_weekly_declines_card', 'nb_monthly_declines_card',
       'hr_nbt/last_30d_av_hr_nbt',
       'day_nbt/last_30d_av_day_nbt', 'last_3d_amt/nbt',
       'year_diff']
    df_cct_not_normalized=df_cct.copy()
    for i in Normalizing_List:
        if np.std(df_cct[i])!=0:
            df_cct[i] = (df_cct[i]-np.mean(df_cct[i]))/np.std(df_cct[i])
        else:
            df_cct[i] = (df_cct[i]-np.mean(df_cct[i])) 
    return df_cct
        
                
        
        
        
    

In [25]:
# Application de la fonction au 2000 parquet -> on obtient les parquets modifies et les parquets avec
    #les fraudes associees pour chaque utilisatuer dans un seul et même dossier 
    
    
for k in range (2000):
    df=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all"+str(k)+".parquet")
    
    df_modified=modification_data_frame(df)
    IsFraud=df_modified["Is Fraud?"].to_frame()
    df_modified=df_modified.drop(["Is Fraud?"],axis = 1)
    
    df_modified.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/df_all'+ str(k)+'.parquet')
    IsFraud.to_parquet('/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/IsFraud'+ str(k)+'.parquet')
    


In [26]:
#on ouvre le parquet 200 pour tester
df200=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_all/df_all200.parquet").head()
df200.head()

,User,Card,Month,Day,Hours,Amount,Use Chip,Card Brand,Card Type,Credit Limit,...,daily_amount,nb_daily_declines_card,nb_weekly_declines_card,nb_monthly_declines_card,bad_pin,insufficient_balance,hr_nbt/last_30d_av_hr_nbt,day_nbt/last_30d_av_day_nbt,last_3d_amt/nbt,Is Fraud?
2418359,200,0,1,1,10,10.14,Chip Transaction,Mastercard,Debit (Prepaid),65.0,...,10.14,0.0,0.0,0.0,0,0,1.000000,1.000000,10.140,0
2418360,200,0,1,1,11,15.30,Chip Transaction,Mastercard,Debit (Prepaid),65.0,...,25.44,0.0,0.0,0.0,0,0,1.333333,1.333333,12.720,0
2418361,200,0,1,2,11,8.64,Chip Transaction,Mastercard,Debit (Prepaid),65.0,...,8.64,0.0,0.0,0.0,0,0,0.750000,1.200000,11.360,0
2418362,200,0,1,2,19,57.08,Chip Transaction,Mastercard,Debit (Prepaid),65.0,...,65.72,0.0,0.0,0.0,0,0,0.800000,1.142857,22.790,0
2418363,200,0,1,3,6,2.61,Chip Transaction,Mastercard,Debit (Prepaid),65.0,...,2.61,0.0,0.0,0.0,0,0,0.833333,1.500000,18.754,0


In [27]:
df200.columns



Index(['User', 'Card', 'Month', 'Day', 'Hours', 'Amount', 'Use Chip',
       'Card Brand', 'Card Type', 'Credit Limit', 'year_diff', 'delta_t_s',
       'delta_t_s_card', 'day_of_week', 'amt/daily_income', 'Retired',
       'daily_amount', 'nb_daily_declines_card', 'nb_weekly_declines_card',
       'nb_monthly_declines_card', 'bad_pin', 'insufficient_balance',
       'hr_nbt/last_30d_av_hr_nbt', 'day_nbt/last_30d_av_day_nbt',
       'last_3d_amt/nbt', 'Is Fraud?'],
      dtype='object')

In [21]:
print(df200.describe())

        User  Card  Month      Day      Hours     Amount  Credit Limit  \
count    5.0   5.0    5.0  5.00000   5.000000   5.000000           5.0   
mean   200.0   0.0    1.0  1.80000  11.400000  18.754000          65.0   
std      0.0   0.0    0.0  0.83666   4.722288  21.897118           0.0   
min    200.0   0.0    1.0  1.00000   6.000000   2.610000          65.0   
25%    200.0   0.0    1.0  1.00000  10.000000   8.640000          65.0   
50%    200.0   0.0    1.0  2.00000  11.000000  10.140000          65.0   
75%    200.0   0.0    1.0  2.00000  11.000000  15.300000          65.0   
max    200.0   0.0    1.0  3.00000  19.000000  57.080000          65.0   

       year_diff     delta_t_s  delta_t_s_card  ...  daily_amount  \
count        5.0      5.000000        5.000000  ...      5.000000   
mean         0.0  31800.000000    31800.000000  ...     22.510000   
std          0.0  34745.206864    34745.206864  ...     25.578765   
min          0.0      0.000000        0.000000  ...      

In [28]:
#on ouvre le parquet 200 pour tester
df200_modified=pd.read_parquet("/Users/vanheuverswyn/Desktop/ProjetStatapp/parquet_modified_all/df_all200.parquet")
print(df200_modified.describe())

        User        Amount  Credit Limit  year_diff     delta_t_s  \
count  110.0  1.100000e+02         110.0      110.0  1.100000e+02   
mean   200.0  3.734387e-17           0.0        0.0  1.715799e-17   
std      0.0  1.004577e+00           0.0        0.0  1.004577e+00   
min    200.0 -3.449111e+00           0.0        0.0 -9.903123e-01   
25%    200.0 -5.688261e-01           0.0        0.0 -7.760909e-01   
50%    200.0 -3.831703e-01           0.0        0.0 -3.564786e-01   
75%    200.0  1.017397e+00           0.0        0.0  4.641037e-01   
max    200.0  2.047622e+00           0.0        0.0  4.305371e+00   

       delta_t_s_card  amt/daily_income  Retired  daily_amount  \
count    1.100000e+02      1.100000e+02    110.0  1.100000e+02   
mean     1.715799e-17      1.261617e-16      0.0 -4.143150e-16   
std      1.004577e+00      1.004577e+00      0.0  1.004577e+00   
min     -9.903123e-01     -3.449111e+00      0.0 -8.277496e-01   
25%     -7.760909e-01     -5.688261e-01      0.0

In [ ]:
#fonction qui applique dbScann au dataFrame A finir

def dbScann (df_cct):
    X = df_cct.values
    X.astype(np.float64)
    #eps min_sample et n_jobs a déterminer  (fixer le meme pour tous ou automatiser une selection?)
    clustering=cluster.DBSCAN(eps=0.8, min_samples=5, n_jobs = -1)
    clustering.fit(X)
    
def PCS(df_cct):    
    pca = PCA()
    pca.fit(df_cct)
    df_cct.columns = df_cct.columns.astype("str")
    df_cct_pca = pca.transform(df_cct)
    df_cct_pca_df = pd.DataFrame({
    "Dim1" : df_cct_pca[:,0], 
    "Dim2" : df_cct_pca[:,1]})
    df_cct_pca_df["labels"] = clustering.labels_
